# Noise Field

In this workshop, we will learn about creation of noise field, based on a set of noise sources.

## 0. Initialization

### 0.1. Load required libraries

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import scipy as sp

### 0.2. Load the envelope lattice as the avialbility lattice

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

### 0.3. Load noise sources

In [3]:
# loading program (agents information) from CSV
noise_source_path = os.path.relpath('../data/noise_points.csv')
noise_sources = np.genfromtxt(noise_source_path, delimiter=',')
noise_sources

array([[ 181.11863812,   95.76447471,    0.        ],
       [ 150.66350669,   94.85490038,    0.        ],
       [ 120.20837525,   93.94532604,    0.        ],
       [  89.75324382,   93.0357517 ,    0.        ],
       [  59.29811239,   92.12617736,    0.        ],
       [  28.84298096,   91.21660303,    0.        ],
       [  -1.61215047,   90.30702869,    0.        ],
       [ -32.0672819 ,   89.39745435,    0.        ],
       [ -62.52241333,   88.48788001,    0.        ],
       [ -92.97754477,   87.57830567,    0.        ],
       [-123.4326762 ,   86.66873134,    0.        ]])

### 0.4. Visualize noise source points

In [4]:
p = pv.Plotter(notebook=True)


# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()

p.add_mesh(noise_sources, point_size=10)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(452.0735071069646, 408.6127635037751, 443.2305261469637),
 (28.8429809600009, -14.61776264318862, 20.0),
 (0.0, 0.0, 1.0)]

## 1. Creation of Noise Field

### 1.1. Computing noise lattices

In [5]:
# create full lattice
full_lattice = avail_lattice * 0 + 1

# extract the coordiantes of the centroid of all voxel
vox_centroids = full_lattice.centroids

# extract voxel indices of all voxels
vox_indices = np.array(np.where(full_lattice==1)).T

# setting the noise base pressure level 
noise_base = 75.0

# initializing the sum lattice of noise
sum_noise_lats = avail_lattice * 0.0

# for each source of noise
for noise_src in noise_sources:
    # initialize the occupation lattice
    dist_latice = avail_lattice * 0.0

    for cen, ind in zip(vox_centroids, vox_indices):
        # compute the euclidian distance
        dist_latice[tuple(ind)] = sp.spatial.distance.euclidean(cen, noise_src)

    # computing the noise lattice from dist lattice
    noise_latice = noise_base - 20 * np.log10(dist_latice) - 8
    
    # summing
    sum_noise_lats += np.power(10, noise_latice / 10.0)


# computing the final aggregation
agg_noise_lats = 10 * np.log10(sum_noise_lats)

### 1.2. Visualizing the noise lattices

In [6]:
# initiating the plotter
p = pv.Plotter(notebook=True)

vis_lattice = agg_noise_lats

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = vis_lattice.shape
# The bottom left corner of the data set
grid.origin = vis_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = vis_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Noise"] = vis_lattice.flatten(order="F")  # Flatten the Lattice

    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm" ,opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(268.15990242089225, 183.15990242089225, 253.15990242089225),
 (35.0, -50.0, 20.0),
 (0.0, 0.0, 1.0)]

### Credits

In [7]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Noise Fields"